# Process @jongraz tiktok stats for Noodle videos

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read in our hand collected list of Noodle videos

In [3]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Create an ID field from the URL

In [4]:
src['id'] = src['url'].str.replace('https://www.tiktok.com/@jongraz/video/', "", regex=False).astype(str)

### Make a list for subsetting the dataframe later

In [5]:
bones_videos = src['id'].to_list()

### Have to run this manually -- for now

In [6]:
# Get cookie and run this...

In [7]:
# !python '/Users/stiles/github/noodle-tracker/_workspace/grab_user_stats.py'

---

### Read scraped stats data

In [8]:
df = pd.read_csv('data/processed/jongraz_videos.csv', dtype={'video_id':str})

### Clean update dates, times

In [9]:
df['timestamp'] = pd.to_datetime(df['video_time'], unit='s')
df['date'] = pd.to_datetime(df['video_time'], unit='s').dt.date
df['month'] = pd.to_datetime(df['video_time'], unit='s').dt.month
df['weekday'] = pd.to_datetime(df['video_time'], unit='s').dt.day_name()
df['time'] = pd.to_datetime(df['video_time'], unit='s').dt.time
df['weekstart_timestamp'] = pd.to_datetime(df['video_time'], unit='s') - pd.to_datetime(df['video_time'], unit='s').dt.weekday * np.timedelta64(1, 'D')
df['weekstart'] = df['weekstart_timestamp'].dt.date
df['updated_date'] = pd.to_datetime("today")

### How many videos? 

In [10]:
len(df)

167

### Select only "bones" videos from @jongraz's feed

In [11]:
bones_df = df[df['video_id'].isin(bones_videos)]

### How many are Noodle videos? 

In [12]:
len(bones_df)

70

In [13]:
bones_df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
0,jongraz,7053837902991658287,his message today is…CLEAR 🦴🔮🦴,1642349620,59,2131,46300,1150,181900,https://www.tiktok.com/@jongraz/video/7053837902991658287?lang=en,2022-01-16 16:13:40,2022-01-16,1,Sunday,16:13:40,2022-01-10 16:13:40,2022-01-10,2022-01-16 12:01:20.864408
5,jongraz,7052341674634235183,your midweek reading! have a great day everyone! 🔮🦴🔮,1642001259,50,2669,66300,1280,318700,https://www.tiktok.com/@jongraz/video/7052341674634235183?lang=en,2022-01-12 15:27:39,2022-01-12,1,Wednesday,15:27:39,2022-01-10 15:27:39,2022-01-10,2022-01-16 12:01:20.864408
7,jongraz,7051972968347503919,plz plan/adjust your day accordingly #bones #nobones #noodletok,1641915407,58,3384,87900,2574,364800,https://www.tiktok.com/@jongraz/video/7051972968347503919?lang=en,2022-01-11 15:36:47,2022-01-11,1,Tuesday,15:36:47,2022-01-10 15:36:47,2022-01-10,2022-01-16 12:01:20.864408
10,jongraz,7050491114775842095,we must HEED THE BONES!! 🔮🦴🔮,1641570386,45,4242,100000,1714,464400,https://www.tiktok.com/@jongraz/video/7050491114775842095?lang=en,2022-01-07 15:46:26,2022-01-07,1,Friday,15:46:26,2022-01-03 15:46:26,2022-01-03,2022-01-16 12:01:20.864408
12,jongraz,7048998918729764102,noodle is…is he all of us today? I think he may be #noodletok #mondaymotivation,1641222956,46,5578,144700,1923,650200,https://www.tiktok.com/@jongraz/video/7048998918729764102?lang=en,2022-01-03 15:15:56,2022-01-03,1,Monday,15:15:56,2022-01-03 15:15:56,2022-01-03,2022-01-16 12:01:20.864408


---

### Download cover art

In [14]:
# for v,u in zip(bones_df['video_cover'], bones_df['video_id']):
#     !wget {v} -O ../assets/images/noodle_covers/{u}

---

### Weekly plays

In [15]:
weekly_plays = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_plays':sum}).reset_index()
weekly_plays.dtypes

weekstart    object
n_plays       int64
dtype: object

In [16]:
weekly_plays['weekstart'] = pd.to_datetime(weekly_plays['weekstart'])

In [17]:
alt.Chart(weekly_plays).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_plays'
).properties(width=630)

alt.Chart(...)

---

### Weekly likes

In [18]:
bones_df.head()

,user_name,video_id,video_desc,video_time,video_duration,n_shares,n_likes,n_comments,n_plays,video_link,timestamp,date,month,weekday,time,weekstart_timestamp,weekstart,updated_date
0,jongraz,7053837902991658287,his message today is…CLEAR 🦴🔮🦴,1642349620,59,2131,46300,1150,181900,https://www.tiktok.com/@jongraz/video/7053837902991658287?lang=en,2022-01-16 16:13:40,2022-01-16,1,Sunday,16:13:40,2022-01-10 16:13:40,2022-01-10,2022-01-16 12:01:20.864408
5,jongraz,7052341674634235183,your midweek reading! have a great day everyone! 🔮🦴🔮,1642001259,50,2669,66300,1280,318700,https://www.tiktok.com/@jongraz/video/7052341674634235183?lang=en,2022-01-12 15:27:39,2022-01-12,1,Wednesday,15:27:39,2022-01-10 15:27:39,2022-01-10,2022-01-16 12:01:20.864408
7,jongraz,7051972968347503919,plz plan/adjust your day accordingly #bones #nobones #noodletok,1641915407,58,3384,87900,2574,364800,https://www.tiktok.com/@jongraz/video/7051972968347503919?lang=en,2022-01-11 15:36:47,2022-01-11,1,Tuesday,15:36:47,2022-01-10 15:36:47,2022-01-10,2022-01-16 12:01:20.864408
10,jongraz,7050491114775842095,we must HEED THE BONES!! 🔮🦴🔮,1641570386,45,4242,100000,1714,464400,https://www.tiktok.com/@jongraz/video/7050491114775842095?lang=en,2022-01-07 15:46:26,2022-01-07,1,Friday,15:46:26,2022-01-03 15:46:26,2022-01-03,2022-01-16 12:01:20.864408
12,jongraz,7048998918729764102,noodle is…is he all of us today? I think he may be #noodletok #mondaymotivation,1641222956,46,5578,144700,1923,650200,https://www.tiktok.com/@jongraz/video/7048998918729764102?lang=en,2022-01-03 15:15:56,2022-01-03,1,Monday,15:15:56,2022-01-03 15:15:56,2022-01-03,2022-01-16 12:01:20.864408


In [19]:
weekly_likes = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_likes':sum}).reset_index()
weekly_likes.dtypes

weekstart    object
n_likes       int64
dtype: object

In [20]:
weekly_likes['weekstart'] = pd.to_datetime(weekly_likes['weekstart'])

In [21]:
alt.Chart(weekly_likes).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_likes'
).properties(width=630)

alt.Chart(...)

---

### Weekly comments

In [22]:
weekly_comments = bones_df[bones_df['timestamp'] > '2021/07/31'].groupby(['weekstart']).agg({'n_comments':sum}).reset_index()
weekly_comments.dtypes

weekstart     object
n_comments     int64
dtype: object

In [23]:
weekly_comments['weekstart'] = pd.to_datetime(weekly_comments['weekstart'])

In [24]:
alt.Chart(weekly_comments).mark_bar(width=20).encode(
    x='weekstart:T',
    y='n_comments'
).properties(width=630)

alt.Chart(...)

---

### Exports

In [25]:
weekly_plays.to_csv("../_data/weekly_plays.csv", index=False)
weekly_plays.to_csv("../assets/data/weekly_plays.csv", index=False)

In [26]:
weekly_likes.to_csv("../_data/weekly_likes.csv", index=False)
weekly_likes.to_csv("../assets/data/weekly_likes.csv", index=False)

In [27]:
weekly_comments.to_csv("../_data/weekly_comments.csv", index=False)
weekly_comments.to_csv("../assets/data/weekly_comments.csv", index=False)